In [1]:
%load_ext autoreload
%autoreload 2

from fractions import Fraction as frac

from ms3 import Parse
from ms3.utils import transform, roman_numeral2fifths, roman_numeral2semitones, name2fifths, rel2abs_key, labels2global_tonic, resolve_relative_keys
from plotly.offline import plot #init_notebook_mode, iplot
#import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [2]:
folder = '~/couperin_concerts/harmonies'
p = Parse(folder, file_re='tsv$')
p.parse_tsv()
p

84 files.
KEY -> EXTENSIONS
-----------------
.   -> {'.tsv': 84}

All 84 tabular files have been parsed, 84 of them as Annotations object(s).
KEY -> ANNOTATION LAYERS
------------------------
.   -> staff  voice  label_type  color  
    -> 1      1      0 (dcml)    default    8414

In [3]:
md = pd.read_csv('~/couperin_concerts/metadata.tsv', sep='\t', index_col=1)
md.head()

,rel_paths,last_mc,last_mn,KeySig,TimeSig,label_count,harmony_version,annotated_key,annotators,reviewers,composer,workTitle,movementNumber,movementTitle,workNumber,poet,lyricist,arranger,copyright,creationDate,mscVersion,platform,source,translator,musescore,ambitus,composed_end,composed_start,originalFormat,staff_1_ambitus,staff_1_instrument,staff_2_ambitus,staff_2_instrument,staff_3_ambitus,staff_3_instrument
fnames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
c01n01_prelude,MS3,25,23,1: 1,1: 4/4,93,2.1.0,G,Eva-Maria Hamberger,Johannes Menke,François Couperin,NaN,NaN,Concert Royal no 1: Prélude,NaN,NaN,NaN,NaN,"© Les Éditions Outremontaises, 2006",2019-10-09,3.02,Linux,NaN,NaN,3.6.2,36-84 (C2-C6),1722,1722,xml,59-84 (B3-C6),Instrument 1,36-57 (C2-A3),Instrument 1,NaN,NaN
c01n02_allemande,MS3,20,18,1: 1,1: 4/4,76,2.1.0,G,Eva-Maria Hamberger,Johannes Menke,François Couperin,NaN,NaN,Concert Royal no 1: Allemande,NaN,NaN,NaN,NaN,"© Les Éditions Outremontaises, 2006",2019-10-09,3.02,Linux,NaN,NaN,3.6.2,31-83 (G1-B5),1722,1722,xml,59-83 (B3-B5),Instrument 1,31-66 (G1-F#4),Instrument 1,NaN,NaN
c01n03_sarabande,MS3,30,28,1: -2,1: 3/4,67,2.1.0,g,Eva-Maria Hamberger,Johannes Menke,François Couperin,NaN,NaN,Concert Royal no 1: Sarabande,NaN,NaN,NaN,NaN,"© Les Éditions Outremontaises, 2006",2019-10-09,3.02,Linux,NaN,NaN,3.6.2,31-81 (G1-A5),1722,1722,xml,58-81 (Bb3-A5),Instrument 1,31-67 (G1-G4),Instrument 1,NaN,NaN
c01n04_gavotte,MS3,18,14,1: -2,1: 2/2,52,2.1.0,g,Eva-Maria Hamberger,Johannes Menke,François Couperin,NaN,NaN,Concert Royal no 1: Gavotte,NaN,NaN,NaN,NaN,"© Les Éditions Outremontaises, 2006",2019-10-09,3.02,Linux,NaN,NaN,3.6.2,38-79 (D2-G5),1722,1722,xml,58-79 (Bb3-G5),Instrument 1,38-57 (D2-A3),Instrument 1,NaN,NaN
c01n05_gigue,MS3,33,30,1: 1,1: 6/8,134,2.1.0,G,Eva-Maria Hamberger,Johannes Menke,François Couperin,NaN,NaN,Concert Royal no 1: Gigue,NaN,NaN,NaN,NaN,"© Les Éditions Outremontaises, 2006",2019-10-09,3.02,Linux,NaN,NaN,3.6.2,38-83 (D2-B5),1722,1722,xml,59-83 (B3-B5),Instrument 1,38-64 (D2-E4),Instrument 1,NaN,NaN


In [4]:
def create_gantt(d, task_column='Task', title='Gantt chart', lines=None, cadences=None):
    """Creates and returns ``fig`` and populates it with features.

    When plotted with plot() or iplot(), ``fig`` shows a Gantt chart representing
    the piece's tonalities as extracted by the class Keys().

    Parameters
    ----------
    d: pd.Dataframe
        DataFrame with at least the columns ['Start', 'Finish', 'Task', 'Resource'].
        Other columns can be selected as 'Task' by passing ``task_column``. 
        Further possible columns: 'Description'
    task_column : str
        If ``d`` doesn't have a 'Task' column, pass the name of the column that you want to use as such.
    title: str
        Title to be plotted

    Examples
    --------

    >>> iplot(create_gantt(df))

    does the same as

    >>> fig = create_gantt(df)
    >>> iplot(fig)

    To save the chart to a file instead of displaying it directly, use

    >>> plot(fig,filename="filename.html")
    """

    colors = {'applied': 'rgb(228,26,28)', # 'rgb(220, 0, 0)',
              'local': 'rgb(55,126,184)',  # (1, 0.9, 0.16),
              'tonic of adjacent applied chord(s)': 'rgb(77,175,74)'} # 'rgb(0, 255, 100)'}
    # 'Bluered', 'Picnic', 'Viridis', 'Rainbow'
    
    if task_column != 'Task':
        d = d.rename(columns={task_column: 'Task'})


    fig = ff.create_gantt(d,colors=colors,group_tasks=True,index_col='Resource',show_colorbar=True,
                       showgrid_x=True, showgrid_y=True ,title=title)

    fig['layout']['xaxis'].update({'type': None, 'title': 'Measures'})
    fig['layout']['yaxis'].update({'title': 'Tonicized keys'})
    
    if lines is not None:
        linestyle = {'color':'rgb(0, 0, 0)','width': 0.2,'dash': 'longdash'}
        lines = [{'type': 'line','x0':position,'y0':0,'x1':position,'y1':20,'line':linestyle} for position in lines]
        fig['layout']['shapes'] = fig['layout']['shapes'] + tuple(lines)
        
            

    if cadences is not None:
        lines = []
        annos = []
        hover_x = []
        hover_y = []
        hover_text = []
        alt = 0
        for i,r in cadences.iterrows():
            m = r.m
            c = r.type
            try:
                key = r.key
            except:
                key = None

            if c == 'PAC':
                c = 'PC'
                w = 1
                d = 'solid'
            elif c == 'IAC':
                c = 'IC'
                w = 0.5
                d = 'solid'
            elif c == 'HC':
                w = 0.5
                d = 'dash'
            elif c == 'EVCAD':
                c = 'EC'
                w = 0.5
                d = 'dashdot'
            elif c == 'DEC':
                c = 'DC'
                w = 0.5
                d = 'dot'
            else:
                print(f"{c}: Kadenztyp nicht vorgesehen")
            #c = c + f"<br>{key}"
            linestyle = {'color':'rgb(55, 128, 191)','width': w,'dash':d}
            annos.append({'x':m,'y':-0.01+alt*0.03,'font':{'size':7},'showarrow':False,'text':c,'xref':'x','yref':'paper'})
            lines.append({'type': 'line','x0':m,'y0':0,'x1':m,'y1':20,'line':linestyle})
            alt = 0 if alt else 1
            hover_x.append(m)
            hover_y.append(-0.5 - alt * 0.5)
            text = "Cad: " + r.type
            if key is not None:
                text += "<br>Key: " + key
            text += "<br>Beat: " + str(r.beat)
            hover_text.append(text)



        fig['layout']['shapes'] = fig['layout']['shapes'] + tuple(lines)
        fig['layout']['annotations'] = annos

        hover_trace=dict(type='scatter',opacity=0,
                        x=hover_x,
                        y=hover_y,
                        marker= dict(size= 14,
                                    line= dict(width=1),
                                    color= 'red',
                                    opacity= 0.3),
                        name= "Cadences",
                        text= hover_text)
        #fig['data'].append(hover_trace)
        fig.add_traces([hover_trace])
    return fig

In [5]:
#create_gantt(make_gantt_data(at), task_column='semitones', lines=phrases)

In [6]:
def make_gantt_data(at, last_mn=None, relativeroots=True):
    """ Uses: rel2abs_key, resolve_relative_keys, roman_numeral2fifths roman_numerals2semitones, labels2global_tonic
    """
    at = at[at.numeral.notna() & (at.numeral != '@none')].copy()
    if 'mn_fraction' not in at.columns:
        mn_fraction = (at.mn + (at.mn_onset.astype(float)/at.timesig.map(frac).astype(float))).astype(float)
        at.insert(at.columns.get_loc('mn')+1, 'mn_fraction', mn_fraction)
    if last_mn is None:
        last_mn = at.mn.max()
    at.sort_values('mn_fraction', inplace=True)
    interval_breaks = at.mn_fraction.append(pd.Series(last_mn+1.0), ignore_index=True)
    at.index = pd.IntervalIndex.from_breaks(interval_breaks, closed='left')
    
    key_groups = at.loc[at.localkey != at.localkey.shift(), ['mn_fraction', 'localkey', 'globalkey', 'globalkey_is_minor']].rename(columns={'mn_fraction': 'Start'})
    key_groups['numeral'] = key_groups.localkey
    key_groups.insert(2, 'semitones', transform(key_groups, roman_numeral2semitones, ['numeral', 'globalkey_is_minor']))
    key_groups.insert(2, 'fifths', transform(key_groups, roman_numeral2fifths, ['numeral', 'globalkey_is_minor']))
    interval_breaks = key_groups.Start.append(pd.Series(last_mn+1.0), ignore_index=True)
    iix = pd.IntervalIndex.from_breaks(interval_breaks, closed='left')
    key_groups.index = iix
    insert_pos = key_groups.columns.get_loc('Start')+1
    key_groups.insert(insert_pos, 'Resource', 'local')
    key_groups.insert(insert_pos, 'Duration', iix.length)
    key_groups.insert(insert_pos, 'Finish', iix.right)
    
    if not relativeroots or at.relativeroot.isna().all():
        return key_groups
    
    levels = list(range(at.index.nlevels))
    def select_groups(df):
        nonlocal levels
        has_applied = df.Resource.notna()
        if has_applied.any():
            df.Resource.fillna('tonic of adjacent applied chord(s)', inplace=True)
            df.relativeroot = df.relativeroot.where(has_applied, df.numeral)
            df['subgroup'] = df.Resource != df.Resource.shift()
            return df
        else:
            return pd.DataFrame(columns=levels).set_index(levels, drop=True)
        
    def gantt_data(df):
        frst = df.iloc[[0]]
        start, finish = df.index[0].left, df.index[-1].right
        frst['Start'] = start
        frst['Finish'] = finish
        frst['Duration'] = finish - start
        frst.index = pd.IntervalIndex.from_tuples([(start, finish)], closed='left')
        return frst
    
    key_groups['abs_numeral'] = key_groups.localkey
    global_numerals = labels2global_tonic(at).numeral
    at['Resource'] = pd.NA
    at.Resource = at.Resource.where(at.relativeroot.isna(), 'applied')
    at['relativeroot_resolved'] = transform(at, resolve_relative_keys, ['relativeroot', 'localkey_is_minor'])
    at['abs_numeral'] = transform(at, rel2abs_key, ['relativeroot_resolved', 'localkey', 'globalkey_is_minor'])
    at.abs_numeral = at.abs_numeral.where(at.abs_numeral.notna(), global_numerals)
    #print(global_numerals)
    #print(at.abs_numeral)
    at['fifths'] = transform(at, roman_numeral2fifths, ['abs_numeral', 'globalkey_is_minor'])
    at['semitones'] = transform(at, roman_numeral2semitones, ['abs_numeral', 'globalkey_is_minor'])
    # using the semitones column includes adjacent variant labels;
    # if only labels of the same mode are to be included, use the numeral column
    adjacent_groups = (at.semitones != at.semitones.shift()).cumsum()
    try:
        at = at.groupby(adjacent_groups, group_keys=False).apply(select_groups).astype({'semitones': int, 'fifths': int})
    except:
        print(at.groupby(adjacent_groups, group_keys=False).apply(select_groups))
        raise
    at.subgroup = at.subgroup.cumsum()
    at = at.groupby(['subgroup', 'localkey'], group_keys=False).apply(gantt_data)
    res = pd.concat([key_groups, at])[['Start', 'Finish', 'Duration', 'Resource', 'abs_numeral', 'fifths', 'semitones', 'localkey', 'globalkey', 'relativeroot']]
    res[['Start', 'Finish', 'Duration']] = res[['Start', 'Finish', 'Duration']].round(2)
    res['Description'] = 'Duration: ' + res.Duration.astype(str) + '<br>Tonicized key: ' + res.abs_numeral + ('<br>In context of localkey ' + res.localkey + ': ' + res.relativeroot).fillna('')
    return res

def get_phraseends(at):
    if 'mn_fraction' not in at.columns:
        mn_fraction = at.mn + (at.mn_onset.astype(float)/at.timesig.map(frac).astype(float))
        at.insert(at.columns.get_loc('mn')+1, 'mn_fraction', mn_fraction)
    return at.loc[at.phraseend.notna(), 'mn_fraction'].to_list()


In [7]:
i = 7
fname = p.fnames['.'][i]
metadata = md.loc[fname]
last_mn = metadata.last_mn
globalkey = metadata.annotated_key
at = p._parsed_tsv[('.', i)]
make_gantt_data(at, last_mn=last_mn, relativeroots=True)
#labels2global_tonic(at, inplace=True)

,Start,Finish,Duration,Resource,abs_numeral,fifths,semitones,localkey,globalkey,relativeroot,Description
"[0.16666666666666666, 5.0)",0.17,5.00,4.83,local,I,0,0,I,G,NaN,Duration: 4.83<br>Tonicized key: I
"[5.0, 10.5)",5.00,10.50,5.50,local,V,1,7,V,G,NaN,Duration: 5.5<br>Tonicized key: V
"[10.5, 12.0)",10.50,12.00,1.50,local,I,0,0,I,G,NaN,Duration: 1.5<br>Tonicized key: I
"[12.0, 15.5)",12.00,15.50,3.50,local,vi,3,9,vi,G,NaN,Duration: 3.5<br>Tonicized key: vi
"[15.5, 16.5)",15.50,16.50,1.00,local,V,1,7,V,G,NaN,Duration: 1.0<br>Tonicized key: V
"[16.5, 17.5)",16.50,17.50,1.00,local,IV,-1,5,IV,G,NaN,Duration: 1.0<br>Tonicized key: IV
"[17.5, 19.333333333333332)",17.50,19.33,1.83,local,ii,2,2,ii,G,NaN,Duration: 1.83<br>Tonicized key: ii
"[19.333333333333332, 31.0)",19.33,31.00,11.67,local,I,0,0,I,G,NaN,Duration: 11.67<br>Tonicized key: I
"[19.833333333333332, 20.0)",19.83,20.00,0.17,applied,V,1,7,I,G,V,Duration: 0.17<br>Tonicized key: V<br>In conte...
"[20.0, 20.5)",20.00,20.50,0.50,tonic of adjacent applied chord(s),V,1,7,I,G,V,Duration: 0.5<br>Tonicized key: V<br>In contex...


In [8]:
p.parsed_tsv.head()

paths     types
rel_paths fnames                                            
.         c01n01_prelude      ./c01n01_prelude.tsv  expanded
          c01n02_allemande  ./c01n02_allemande.tsv  expanded
          c01n03_sarabande  ./c01n03_sarabande.tsv  expanded
          c01n04_gavotte      ./c01n04_gavotte.tsv  expanded
          c01n05_gigue          ./c01n05_gigue.tsv  expanded

In [9]:
p._parsed_tsv.keys()

dict_keys([('.', 0), ('.', 1), ('.', 2), ('.', 3), ('.', 4), ('.', 5), ('.', 6), ('.', 7), ('.', 8), ('.', 9), ('.', 10), ('.', 11), ('.', 12), ('.', 13), ('.', 14), ('.', 15), ('.', 16), ('.', 17), ('.', 18), ('.', 19), ('.', 20), ('.', 21), ('.', 22), ('.', 23), ('.', 24), ('.', 25), ('.', 26), ('.', 27), ('.', 28), ('.', 29), ('.', 30), ('.', 31), ('.', 32), ('.', 33), ('.', 34), ('.', 35), ('.', 36), ('.', 37), ('.', 38), ('.', 39), ('.', 40), ('.', 41), ('.', 42), ('.', 43), ('.', 44), ('.', 45), ('.', 46), ('.', 47), ('.', 48), ('.', 49), ('.', 50), ('.', 51), ('.', 52), ('.', 53), ('.', 54), ('.', 55), ('.', 56), ('.', 57), ('.', 58), ('.', 59), ('.', 60), ('.', 61), ('.', 62), ('.', 63), ('.', 64), ('.', 65), ('.', 66), ('.', 67), ('.', 68), ('.', 69), ('.', 70), ('.', 71), ('.', 72), ('.', 73), ('.', 74), ('.', 75), ('.', 76), ('.', 77), ('.', 78), ('.', 79), ('.', 80), ('.', 81), ('.', 82), ('.', 83)])

In [ ]:
USE = 'semitones' # choose from 'semitones', 'fifths', 'numeral'
for i, fname in enumerate(p.fnames['.']):
    print(i, fname)
    metadata = md.loc[fname]
    last_mn = metadata.last_mn
    globalkey = metadata.annotated_key
    at = p._parsed_tsv[('.', i)]
    data = make_gantt_data(at, last_mn=last_mn, relativeroots=True)
    phrases = get_phraseends(at)
    data.sort_values(USE, ascending=False, inplace=True)
    fig = create_gantt(data, title=f"{fname} ({globalkey})", task_column=USE, lines=phrases)
    plot(fig, filename=f'docs/coup_gantt/{fname}.html')

0 c04n02_allemande
1 c06n04_air_diable
2 c09n05_vivacite
3 c10n04_tromba
4 c11n01_majestueusement
5 c14n02_allemande
6 c14n03_sarabande
7 c01n05_gigue
8 c07n02_allemande
9 c04n06_rigaudon
10 c09n06_Sarabande
11 c08n01_ouverture
12 c11n06_sarabande
13 parnasse_03
14 parnasse_02
15 c05n02_allemande
16 c05n01_prelude
17 parnasse_06
18 c10n01_gravement
19 c09n01_charme
20 c06n02_allemande
21 c10n02_air
22 c03n03_courante
23 parnasse_05
24 c09n03_graces
25 c10n03_plainte
26 c04n03_courante_francoise
27 c11n03_seconde_allemande
28 c09n08_caetera
29 c08n09_air_leger
30 c03n06_musette_1
31 c11n02_allemande
32 c08n02_ritournele
33 parnasse_01
34 c08n10_air_lentement
35 c05n04_gavote
36 c05n05_musete
37 c09n02_lenjouement
38 c02n05_echos
39 c07n03_sarabande
40 c07n04_fuguete
41 c03n02_allemande
42 c11n05_seconde_courante
43 c08n06_Loure
44 c11n04_courante
45 c09n07_douceur
46 c14n04_fuguete
47 c04n04_courante_a_litalienne
48 c01n02_allemande
49 parnasse_07
50 c11n07_gigue
51 c07n06_siciliene
52 

In [ ]:
for f in sorted(p.fnames['.']):
    print(f'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="coup_gantt/{f}.html" height="600" width="100%"></iframe>')